# Add TimelinkDatabase.get_group_model(group name). #53


https://github.com/time-link/timelink-py/issues/53

* Timelink Python package version: 1.1.13
* Python version: 3.10, 3.11
* Operating System: MacOs

### Description

it is not always possible to get the ORM corresponding to a group because a group
can extend another one and not have its own mapping. However internally Timelink
knows the model to use, but the end user need to inspect the structure file to figure out.

In str
```
pars name=lugar; source=geoentity
```

in python
```python
db.get_model("lugar") # returns none
```

Or make get_model smarter.

In [1]:
# NBVAL_IGNORE_OUTPUT
from timelink.notebooks import TimelinkNotebook
tlnb = TimelinkNotebook(
    db_name='issue53',
    db_type='sqlite',)

tlnb.print_info()

Timelink version: 1.1.16
Project name: test-project
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Database type: sqlite
Database name: issue53
Kleio image: timelinkserver/kleio-server
Kleio server token: bkZpF...
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Kleio server container: great_austin
Kleio version requested: latest
Kleio server version: 12.6.577 (2024-10-24 16:53:53)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=test-project, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project, db_type=sqlite, db_name=issue53, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, pos

## import a file with groups with no direct mapping

In [2]:
# NBVAL_IGNORE_OUTPUT

import logging
logging.basicConfig(level=logging.INFO)

path = "sources/reference_sources/cronologias"
tlnb.update_from_sources(path=path)
# get the import status
import_status = tlnb.get_import_status(path=path)
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

In [3]:
# NBVAL_IGNORE_OUTPUT

import logging
logging.basicConfig(level=logging.INFO)

path = "sources/reference_sources/devassas"
tlnb.update_from_sources(path=path)
# get the import status
import_status = tlnb.get_import_status(path=path)
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

In [4]:
from timelink.api.models import Entity

for group, model in sorted(Entity.group_models.items()):
    print(f"{group:24} - {model}")

acusa                    - <class 'timelink.api.models.pom_som_mapper.Acusacoes'>
acusada                  - <class 'timelink.api.models.person.Person'>
acusado                  - <class 'timelink.api.models.person.Person'>
caso                     - <class 'timelink.api.models.pom_som_mapper.Caso'>
class                    - <class 'timelink.api.models.rentity.REntity'>
crono                    - <class 'timelink.api.models.pom_som_mapper.Evento'>
devassa                  - <class 'timelink.api.models.act.Act'>
fonte                    - <class 'timelink.api.models.source.Source'>
ls                       - <class 'timelink.api.models.attribute.Attribute'>
lugar                    - <class 'timelink.api.models.geoentity.Geoentity'>
pai                      - <class 'timelink.api.models.person.Person'>
referida                 - <class 'timelink.api.models.person.Person'>
referido                 - <class 'timelink.api.models.person.Person'>
rel                      - <class 'timelink.

In [5]:

from sqlalchemy import select

eventos, lugares, pessoas = tlnb.db.get_model(["crono", "lugar", "person"])

stmt = (select(eventos, lugares)
        .join(lugares,
              eventos.id == lugares.inside)
        .order_by(lugares.name, eventos.the_date))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    for evento, lugar in results:
        print(f"{lugar.name:16} {evento.the_date} {evento.description}")
        in_query = select(pessoas).where(pessoas.inside == evento.id)
        people_in_event = session.execute(in_query).scalars().all()
        for person in sorted(people_in_event, key=str):  # sort on str rep
            print(f"{' ' * 32} {person.groupname} {person.name}")

print()
print(stmt)


Macau            15551120 Chegada primeiros jesuítas a Macau
                                 referido António Dias
                                 referido Belchior Dias
                                 referido Belchior Nunes Barreto
                                 referido Estevão de Góis
                                 referido Fernão Mendes Pinto
                                 referido Gaspar Vilela
Macau            15620000 Embaixada para Pequim em Macau
                                 referido Diogo Pereira
                                 referido Giovanni Battista de Monte
                                 referido Guilherme Pereira
                                 referido Luís Fróis
                                 referido Pedro Quintero
Pequim           15620000 Embaixada para Pequim em Macau
                                 referido Diogo Pereira
                                 referido Giovanni Battista de Monte
                                 referido Guilherme P

## Devassas



In [6]:

from sqlalchemy import select
from sqlalchemy import inspect

casos = tlnb.db.get_model("caso")
acusacoes = tlnb.db.get_model("acusa")

acusados = tlnb.db.get_model("acusado")  # acusado does not work

insp = inspect(casos)
print(insp.columns.keys())

insp = inspect(acusacoes)
print(insp.columns.keys())

insp = inspect(acusados)
print(insp.columns.keys())

['id', 'pom_class', 'inside', 'the_source', 'the_order', 'the_level', 'the_line', 'groupname', 'extra_info', 'updated', 'indexed', 'name', 'the_type', 'obs', 'id', 'obs', 'the_type']
['id', 'pom_class', 'inside', 'the_source', 'the_order', 'the_level', 'the_line', 'groupname', 'extra_info', 'updated', 'indexed', 'name', 'the_type', 'obs', 'id', 'idcaso', 'literal', 'obs', 'origem']
['id', 'pom_class', 'inside', 'the_source', 'the_order', 'the_level', 'the_line', 'groupname', 'extra_info', 'updated', 'indexed', 'id', 'name', 'sex', 'obs']


In [7]:
casos, acusados = tlnb.db.get_model(["caso", "acusado"])

stmt = (select(casos, acusados)
        .join(acusados, acusados.inside == casos.id).order_by(casos.the_line))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    caso_id = None
    for caso,acusados in sorted(results, key=str)[:10]:
        if caso_id != caso.id:
            caso_id = caso.id
            print()
            print(caso.id, caso.obs)
        print(" ",acusados.name)


c1692-ana-simoes-feiticeira ana simoes,2114a
  ana simoes

c1692-antonio-catarina-fernandes tiveram filho(a). 2.o a t.a 23 manuel goncalves, ele foi prezo por ordem do pai dela.
  antonio
  catarina fernandes

c1692-antonio-cordeiro-alcouc dava mulheres em casa. Margem: tirada de ambos 1.1.1691
  antonio cordeiro
  francisca nunes

c1692-antonio-gomes-maria quarto grau margem: tirada dela em 25.1.692 dele 14.5.1693
  antonio gomes
  maria

c1692-antonio-simoes-maria-domingues da filha
  antonio simoes
  maria domingues

c1692-antonio-simoes-raposo-maria 
  antonio simoes raposo


In [8]:


stmt = (select(casos, acusacoes)
        .join(casos,
              acusacoes.idcaso == casos.id)
        .order_by(casos.the_line, acusacoes.the_line))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    for caso, acusacao in sorted(results, key=str)[:10]:
        print(caso.the_type,caso.obs)
        print(acusacao)
        # print(f"{lugar.name:16} {evento.the_date} {evento.description}")

print()
print(stmt)

caso ana simoes,2114a
acusa$d1692-2114-obj137-20/idcaso=c1692-ana-simoes-feiticeira/literal=diz que fez mal a domingos antunes da carvalheira, casado/obs=feitic/id=d1692-2114-obj137-20
caso ana simoes,2114a
acusa$d1692-2116-obj139-20/idcaso=c1692-ana-simoes-feiticeira/literal=t.a e' mulher de uma suposta vitima. Diz que o marido adoeceu e a denunciada apareceu la com a cura e a mulher suspeita que foi ela que o fez adoecer./obs=feitic/id=d1692-2116-obj139-20
caso ana simoes,2114a
acusa$d1692-2117-obj140-20/idcaso=c1692-ana-simoes-feiticeira/obs=feitic/id=d1692-2117-obj140-20
caso ana simoes,2114a
acusa$d1692-2118-obj141-20/idcaso=c1692-ana-simoes-feiticeira/obs=feitic/id=d1692-2118-obj141-20
caso ana simoes,2114a
acusa$d1692-2119-obj142-20/idcaso=c1692-ana-simoes-feiticeira/obs=feitic/id=d1692-2119-obj142-20
caso ana simoes,2114a
acusa$d1692-2122-obj146-20/idcaso=c1692-ana-simoes-feiticeira/obs=feitic/id=d1692-2122-obj146-20
caso ana simoes,2114a
acusa$d1692-2129-obj165-20/idcaso=c1692